In [ ]:
#Import 
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm 
import streamlit as st
from fileHandler import csvHandler, jsonHandler

#Read Json
with open("kode_negara_lengkap.json", "r") as read_file:
    data = json.load(read_file)
# for i in data:
#     print(type(i))
print(data[0])
dfJ = pd.DataFrame(data)

#Read csv
csv = pd.read_csv("produksi_minyak_mentah.csv")
dfC = pd.DataFrame(csv)
print(dfC)

#Remove Kumpulan Negara
list_kodenegarahuruf = []
list_organsisasi = []

for i in list(dfC['kode_negara']):
    if i not in list_kodenegarahuruf:
        list_kodenegarahuruf.append(i)
        
for i in list_kodenegarahuruf:
    if i not in list(dfJ['alpha-3']):
        list_organsisasi.append(i)
        
for i in list_organsisasi:
    df_csv = dfC[dfC.kode_negara != i]
    if i in list_kodenegarahuruf:
        list_kodenegarahuruf.remove(i)
        
#Membuat Data Frame tiap File
st.title('Data Produksi Minyak Mentah')
ch_ = csvHandler('produksi_minyak_mentah.csv')
jh_ = jsonHandler('kode_negara_lengkap.json')
csv_ = ch_.dataFrame
df_info = jh_.dataFrame
negara_li = df_info['name'].tolist()

#Setting Output
st.sidebar.title("Pilihan Pengaturan")
st.sidebar.header('Pengaturan Jumlah Produksi Per Bulan')
left_col, mid_col, right_col = st.columns(3)
negara = st.sidebar.selectbox('Input Pilihan Negara : ',negara_li) 

kode = df_info[df_info['name']==negara]['alpha-3'].tolist()[0]

st.sidebar.write('Kode negara : ',kode)
st.sidebar.write('Negara : ',negara)

#Mengubah String Data menjadi Float
dfJ['produksi'] = dfJ['produksi'].astype(str).str.replace(".", "", regex=True).astype(float)
dfJ['produksi'] = dfJ['produksi'].astype(str).str.replace(",", "", regex=True).astype(float)
dfJ['produksi'] = pd.to_numeric(dfJ['produksi'], errors='coerce')

#Bagian A (Tabel)
df2 = pd.DataFrame(dfC,columns= ['kode_negara','tahun','produksi'])
df2=df2.loc[df2['kode_negara']==kode]
df2['produksi'] = pd.to_numeric(df2['produksi'], errors='coerce')
mid_col.write(df2)

#Bagian A (Grafik)
fig, ax = plt.subplots()
ax.plot(df2['tahun'], df2['produksi'], label = df2['tahun'])
ax.set_title("Jumlah Produksi Per Tahun di Negara Pilihan")
ax.set_xlabel("Tahun", fontsize = 11)
ax.set_ylabel("Jumlah Produksi", fontsize = 11)
ax.legend(fontsize = 3)
plt.show()
st.pyplot(fig)

#Bagian B
st.sidebar.header('Pengaturan Negara dengan Produksi Terbesar')
tahun = st.sidebar.number_input("Pilih Tahun produksi", min_value=1971, max_value=2015)
n = st.sidebar.number_input("Pilih Banyak Negara", min_value=1, max_value=None)

dfb = dfJ.loc[dfJ['tahun'] == tahun][:n]
dfb = dfb.sort_values(by='produksi', ascending = False)
df3 = dfb[:n]

df3.plot.bar(x='kode_negara', y='produksi')
plt.show()
st.pyplot(plt)

#Bagian C
list_a = []
kumulatif = []

for i in list (csv_['kode_negara']) :
    if i not in list_a:
        list_a.append(i)
        
for i in list_a :
    a=csv_.loc[csv_['kode_negara'] ==i,'produksi'].sum()
    kumulatif.append(a)
    
dk = pd.DataFrame(list(zip(list_a,kumulatif)), columns = ['kode_negara','kumulatif'])
dk = dk.sort_values(by=['kumulatif'], ascending = False)
dk = dk[:n]

dk.plot.bar(x='kode_negara', y='kumulatif') 
plt.show()
st.pyplot(plt)

#Bagian D (1)
